In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 496 µs (started: 2021-11-17 14:39:19 -08:00)


# Options Apps Project

## Goal
Creating a suite of equity option web apps which expands on, and was inspired by, [Harshit Tyagi’s](https://www.linkedin.com/in/tyagiharshit/) [article](https://www.kdnuggets.com/2021/09/-structured-financial-newsfeed-using-python-spacy-and-streamlit.html) on building a financial newsfeed app.

## Skills Involved
Web scraping, NLP, options price modeling, predictive modeling of volitility, and building apps with Streamlit.

## Imports

In [2]:
import sys, os, math
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 22.5 s (started: 2021-11-17 14:39:19 -08:00)


## requests: Grabbing Headlines

In [3]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 205 ms (started: 2021-11-17 14:39:42 -08:00)


## bs4: Saving Headlines

In [4]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
print(f'{type(headlines)}\n')
print(headlines)

<class 'bs4.element.ResultSet'>

[<title>MarketWatch.com - Top Stories</title>, <title>MarketWatch.com - Top Stories</title>, <title>What's Worth Streaming: Here’s everything new coming to Hulu in December 2021</title>, <title>Market Extra: Inflation in ‘process of peaking,’ but still poses threat to investor portfolios, says Wells Fargo</title>, <title>Earnings Results: Victoria’s Secret stock soars 12% on profit beat</title>, <title>The Conversation: High-speed internet is as essential as water or electricity, infrastructure bill proclaims</title>, <title>NewsWatch: Nvidia reports record sales and beats on earnings again, sending stock higher</title>, <title>Earnings Results: Nvidia reports record sales and beats on earnings again, sending stock higher</title>, <title>: Cisco stock drops nearly 7% after earnings guidance disappoints</title>, <title>: Rivian stock falls by as much as 18% to mark first stumble since its IPO</title>, <title>Washington Watch: 40% of America’s trucking ca

### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [5]:
# grabbing test headline
test_headline = headlines[2]
print(f'{test_headline}\n')
print(test_headline.text)

<title>What's Worth Streaming: Here’s everything new coming to Hulu in December 2021</title>

What's Worth Streaming: Here’s everything new coming to Hulu in December 2021
time: 1.26 ms (started: 2021-11-17 14:39:42 -08:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [6]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 730 ms (started: 2021-11-17 14:39:42 -08:00)


### Tokenizing String
Tokenizing the test headline.

In [7]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(f'{test_headline}\n')
for token in processed_hline:
  print(token)

<title>What's Worth Streaming: Here’s everything new coming to Hulu in December 2021</title>

What
's
Worth
Streaming
:
Here
’s
everything
new
coming
to
Hulu
in
December
2021
time: 10.9 ms (started: 2021-11-17 14:39:43 -08:00)


## Saving List of Tokenized Headlines

In [8]:
processed_hlines = [nlp(headlines[i].text) for i in range(len(headlines))]
for line in processed_hlines:
    print(line)

MarketWatch.com - Top Stories
MarketWatch.com - Top Stories
What's Worth Streaming: Here’s everything new coming to Hulu in December 2021
Market Extra: Inflation in ‘process of peaking,’ but still poses threat to investor portfolios, says Wells Fargo
Earnings Results: Victoria’s Secret stock soars 12% on profit beat
The Conversation: High-speed internet is as essential as water or electricity, infrastructure bill proclaims
NewsWatch: Nvidia reports record sales and beats on earnings again, sending stock higher
Earnings Results: Nvidia reports record sales and beats on earnings again, sending stock higher
: Cisco stock drops nearly 7% after earnings guidance disappoints
: Rivian stock falls by as much as 18% to mark first stumble since its IPO
Washington Watch: 40% of America’s trucking capacity is left on the table every day, MIT expert tells Congress
Key Words: Why is there no EV love for Ford and GM? Cathie Wood says ‘they don’t have the DNA for this brave new world’ of electric cars

## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [9]:
# pulling company name tokens from headlines
companies = []
for title in processed_hlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Nvidia', 'EV', 'GM', 'MIT', 'Congress', 'Ford', 'Washington Watch', 'Wells Fargo'}
time: 100 ms (started: 2021-11-17 14:39:43 -08:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [10]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 648 ms (started: 2021-11-17 14:39:43 -08:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [11]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking length
print(f'List Length: {len(symbols)} \n')

# checking first 5 symbols
print(f'First five symbols: {symbols[:5]}')

List Length: 505 

First five symbols: ['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 15.7 ms (started: 2021-11-17 14:39:44 -08:00)


The length of the list looks good, but we have extra characters we need to strip from the symbols.

In [12]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 1.15 ms (started: 2021-11-17 14:39:44 -08:00)


Looks good. We can apply the same technique to the second cell of the row to scrape and save the company name, and to the fifth cell to grab the company industry. 

### Names
Grabbing the company name from the second cell of each row.

In [13]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 14.2 ms (started: 2021-11-17 14:39:44 -08:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [14]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 2)


,Company Name,Symbol
0,3M,MMM
1,Abbott Laboratories,ABT
2,AbbVie,ABBV
3,Abiomed,ABMD
4,Accenture,ACN


time: 18.4 ms (started: 2021-11-17 14:39:44 -08:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [15]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 3.2 ms (started: 2021-11-17 14:39:44 -08:00)


In [16]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 2.57 s (started: 2021-11-17 14:39:44 -08:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [17]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 10.4 s (started: 2021-11-17 14:39:47 -08:00)


In [18]:
# checking dict
stock_data

{'Company': ['Nvidia', 'MGM Resorts International', 'Ford', 'Wells Fargo'],
 'Symbol': ['NVDA', 'MGM', 'F', 'WFC'],
 'currentPrice': [292.61, 45.51, 19.96, 49.84],
 'dayHigh': [305.09, 46.2, 20.01, 50.69],
 'dayLow': [288, 45.24, 19.44, 49.53],
 '52wkHigh': [323.1, 51.17, 20.51, 52.57],
 '52wkLow': [115.665, 25.79, 8.43, 24.97],
 'dividendRate': [0.16, 0.01, 0.4, 0.8]}

time: 2.38 ms (started: 2021-11-17 14:39:57 -08:00)


### Data Frame of S&P 500 Stocks in the News

In [19]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,Nvidia,NVDA,292.61,305.09,288.00,323.10,115.665,0.16
1,MGM Resorts International,MGM,45.51,46.20,45.24,51.17,25.790,0.01
2,Ford,F,19.96,20.01,19.44,20.51,8.430,0.40
3,Wells Fargo,WFC,49.84,50.69,49.53,52.57,24.970,0.80


time: 8.65 ms (started: 2021-11-17 14:39:57 -08:00)


In [20]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       4 non-null      object 
 1   Symbol        4 non-null      object 
 2   currentPrice  4 non-null      float64
 3   dayHigh       4 non-null      float64
 4   dayLow        4 non-null      float64
 5   52wkHigh      4 non-null      float64
 6   52wkLow       4 non-null      float64
 7   dividendRate  4 non-null      float64
dtypes: float64(6), object(2)
memory usage: 384.0+ bytes
time: 5.58 ms (started: 2021-11-17 14:39:57 -08:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [21]:
h.get_prices('MMM')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,3M Company,MMM,183.34,183.87,181.63,208.95,163.38,5.92


time: 2.38 s (started: 2021-11-17 14:39:57 -08:00)


In [22]:
h.get_prices('AAPL')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,Apple Inc.,AAPL,153.49,155,150.995,157.26,112.59,0.88


time: 2.32 s (started: 2021-11-17 14:40:00 -08:00)


## Options Markets Tests

In [23]:
# creating a fuction to display available expirations
def get_expirations(symbol : str):
    stock = yf.Ticker(symbol)
    return stock.options

time: 590 µs (started: 2021-11-17 14:40:02 -08:00)


In [24]:
get_expirations('AAPL')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-02-18',
 '2022-03-18',
 '2022-04-14',
 '2022-05-20',
 '2022-06-17',
 '2022-09-16',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

time: 221 ms (started: 2021-11-17 14:40:02 -08:00)


In [25]:
get_expirations('MMM')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-04-14',
 '2022-06-17',
 '2022-07-15',
 '2023-01-20',
 '2024-01-19')

time: 131 ms (started: 2021-11-17 14:40:03 -08:00)
